In [34]:
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
from time import sleep

def get_last_page(url: str) -> str:
    pages = get(url)
    pages_lida = bs(pages.text, 'html.parser')
    links = pages_lida.find('span', {'class': 'wp-pager-pages'}).find_all('a')
    return max([link.get_text() for link in links])

def get_rank(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    rank_tag = ler_pg.find_all('td', {'class':'IndividualStatsRank'})
    ranks = [rank.get_text() for rank in rank_tag]
    return ranks

def get_player(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    player_tag = ler_pg.find_all('a', {'class':'StatsCenterPlayerName'})
    players = [nome.find('span', {'class':'hidden-xs'}).get_text() for nome in player_tag]
    return players

def get_team(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    team_tag = ler_pg.find_all('td', {'class':'StatsCenterTeams'})
    team = []
    for n in team_tag:
        if n.find('span', {'class':'visible-sm-inline'}) != None:
            team.append(n.find('span', {'class':'visible-md-inline'}).get_text())
        else:
            team.append('N/A')
    #team = [n.find('span', {'class':'visible-sm-inline'}).get_text() for n in team_tag]
    return team

def get_games(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    games = [n.get_text()[0:] for n in games_tag[0::4]]
    return games

def get_index(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    index_r = [n.get_text()[0:] for n in games_tag[1::4]]
    return index_r

def get_aver(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    aver = [n.get_text()[0:] for n in games_tag[2::4]]
    return aver

def get_per(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    per_40 = [n.get_text()[0:] for n in games_tag[3::4]]
    return per_40

def get_season(url):
    season = ler_pg.find('select', {'name':'ctl00$ctl00$ctl00$ctl00$maincontainer$maincontent$contentpane$ctl01$ddlSeasonSingle'}).find('option', {'selected':'selected'}).get_text()
    return season

In [35]:
base_url = 'https://www.adidasngt.com/'
stats = f'{base_url}u18/game-center/statistics'

stats_get = get(stats)
stats_page = bs(stats_get.text, 'html.parser')

In [36]:
seasons = stats_page.find('select', {'name':'ctl00$ctl00$ctl00$ctl00$maincontainer$maincontent$contentpane$ctl01$ddlSeasonSingle'}).find_all('option')

season_dict = {season.get_text() : season.get('value')[11:] for season in seasons}

link_seasons = [f'{stats}?mode=Leaders&entity=Players&seasonmode=Single&seasoncode={season}&cat=Valuation&agg=Accumulated' for season in season_dict.values()]

In [62]:
lista_rank = []
lista_player = []
lista_team = []
lista_games = []
lista_index = []
lista_aver = []
lista_per = []
lista_season = []

for url in link_seasons:
    season_pg = get(url)
    season_page = bs(season_pg.text, 'html.parser')
    if season_page.find('span', {'class': 'wp-pager-pages'}):
        last_page = int(get_last_page(url))
        last_url = ['{}&page={}'.format(url, n) for n in range(1, last_page+1)]
        for urls in last_url:
            pagina = get(urls)
            ler_pg = bs(pagina.text, 'html.parser')
            lista_rank += get_rank(urls)
            lista_player += get_player(urls)
            lista_team += get_team(urls)
            lista_games += get_games(urls)
            lista_index += get_index(urls)
            lista_aver += get_aver(urls)
            lista_per += get_per(urls)
            print(urls)
            lista_season += [get_season(urls)] * len(get_rank(urls))
            print(len(get_rank(urls)))
        print(len(lista_season))
print('ACABOU!')

"for url in link_seasons:\n    season_pg = get(url)\n    season_page = bs(season_pg.text, 'html.parser')\n    if season_page.find('span', {'class': 'wp-pager-pages'}):\n        last_page = int(get_last_page(url))\n        last_url = ['{}&page={}'.format(url, n) for n in range(1, last_page+1)]\n        for urls in last_url:\n            pagina = get(urls)\n            ler_pg = bs(pagina.text, 'html.parser')\n            lista_rank += get_rank(urls)\n            lista_player += get_player(urls)\n            lista_team += get_team(urls)\n            lista_games += get_games(urls)\n            lista_index += get_index(urls)\n            lista_aver += get_aver(urls)\n            lista_per += get_per(urls)\n            print(urls)\n            lista_season += [get_season(urls)] * len(get_rank(urls))\n            print(len(get_rank(urls)))\n        print(len(lista_season))\nprint('ACABOU!')"

In [45]:
'''lista_rank = []
lista_player = []
lista_team = []
lista_games = []
lista_index = []
lista_aver = []
lista_per = []'''

listao = zip(lista_season, lista_rank, lista_player, lista_team, lista_games, lista_index, lista_aver, lista_per)

In [60]:
stats_dict = {'season':lista_season, 'rank':lista_rank, 'player':lista_player, 'team':lista_team, 'games':lista_games, 'index':lista_index, 'average':lista_aver, 'per_40min':lista_per}

df = pd.DataFrame(stats_dict)

In [61]:
df.to_csv('PlayersU18.csv')